In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

from scipy.stats import f_oneway, pointbiserialr, chi2_contingency, ttest_ind
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder

from google.colab import drive
drive.mount('/content/drive')
from numpy.random import seed
seed(1)

Mounted at /content/drive


# Cargamos datos:






In [ ]:
scores = pd.read_csv('/content/drive/MyDrive/ARCHIVOS/score.csv', encoding='utf-8',index_col=False)
scores


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/ARCHIVOS/score.csv'

Se asignaron etiquetas a las variables culitativas estado,municipio,colonia y tipo(desarrollo urbano). Para los estados y la colonia simplemente se enumeraron, pero para el caso de municipio se contaron los datos únicos y se ennumeró de esa forma. Por último, para el caso de tipo(desarrollo urbano) se asignaron de la siguiente forma:


1.   Rural
2.   En transición
3.   Semi-urbano
4.   Urbano
5.   Semi-metrópoli
6.   Metrópoli


In [ ]:
#Veamos algunas estadísticas de nuestro DataFrame
scores.describe()


In [ ]:
#Verificamos que nuestro dataframe no contenga valores nulos para tener un óptimo desempeño
scores.info()

# **Matriz de correlación**

In [ ]:
# Calculamos la matriz de correlacion
correlation_matrix = scores.corr(method='pearson')
# Crear la gráfica con el tamaño deseado
fig = px.imshow(correlation_matrix, text_auto=True, aspect="auto",
                labels=dict(color="Correlation"),
                width=1100, height=1100)  # Ajusta el tamaño según tus preferencias

fig.show()

# **Midamos correlación de variables cualitativas (Estado, Municipio y Colonia)**




In [ ]:
#Esta primera parte nos dice la distribución conjunta de la variable objetivo Y con nuestra variable X6
#Básicamente nos cuenta los valores obtenidos para Y por cada valor de nuestra variable X6
CrosstabResult = pd.crosstab(index = scores['Y objetivo'], columns = scores['Estado'])
print(CrosstabResult, '\n')
#El resultado de chi2_contingency contiene varios valores estadísticos, pero aquí estamos interesados en el p-valor.
#Se obtiene mediante ChiSqResult[1]. Un p-valor pequeño (generalmente menor que un umbral predefinido, como 0.05) sugiere que hay una relación significativa entre las dos variables.
#Un p-valor grande sugiere que no hay una relación significativa y, por lo tanto, la hipótesis nula no se rechaza.
ChiSqResult = chi2_contingency(CrosstabResult)
print('P-Value de Estado:', ChiSqResult[1])

In [ ]:
CrosstabResult = pd.crosstab(index = scores['Y objetivo'], columns = scores['Municipio'])
print(CrosstabResult, '\n')
#El resultado de chi2_contingency contiene varios valores estadísticos, pero aquí estamos interesados en el p-valor.
#Se obtiene mediante ChiSqResult[1]. Un p-valor pequeño (generalmente menor que un umbral predefinido, como 0.05) sugiere que hay una relación significativa entre las dos variables.
#Un p-valor grande sugiere que no hay una relación significativa y, por lo tanto, la hipótesis nula no se rechaza.
ChiSqResult = chi2_contingency(CrosstabResult)
print('P-Value de Municipio:', ChiSqResult[1])

In [ ]:
CrosstabResult = pd.crosstab(index = scores['Y objetivo'], columns = scores['Colonia'])
print(CrosstabResult, '\n')
#El resultado de chi2_contingency contiene varios valores estadísticos, pero aquí estamos interesados en el p-valor.
#Se obtiene mediante ChiSqResult[1]. Un p-valor pequeño (generalmente menor que un umbral predefinido, como 0.05) sugiere que hay una relación significativa entre las dos variables.
#Un p-valor grande sugiere que no hay una relación significativa y, por lo tanto, la hipótesis nula no se rechaza.
ChiSqResult = chi2_contingency(CrosstabResult)
print('P-Value de Colonia:', ChiSqResult[1])

In [ ]:
CrosstabResult = pd.crosstab(index = scores['Y objetivo'], columns = scores['Tipo'])
print(CrosstabResult, '\n')
#El resultado de chi2_contingency contiene varios valores estadísticos, pero aquí estamos interesados en el p-valor.
#Se obtiene mediante ChiSqResult[1]. Un p-valor pequeño (generalmente menor que un umbral predefinido, como 0.05) sugiere que hay una relación significativa entre las dos variables.
#Un p-valor grande sugiere que no hay una relación significativa y, por lo tanto, la hipótesis nula no se rechaza.
ChiSqResult = chi2_contingency(CrosstabResult)
print('P-Value de Tipo de desarrollo urbano:', ChiSqResult[1])

# Haremos un mapeo de la distribución de algunas de las variables para conocer cómo es dicha distribución

In [ ]:
plt.figure(figsize=(20,8))

ax1 = plt.subplot(2,4,1)
ax2 = plt.subplot(2,4,2)
ax3 = plt.subplot(2,4,3)
ax4 = plt.subplot(2,4,4)
ax5 = plt.subplot(2,4,5)

ax1.hist(scores['Etiqueta tipo'], bins=30, color='green',edgecolor='purple', alpha=0.5)
ax1.set_xlabel('Tipo de desarrollo urbano ', size=15)
ax1.set_ylabel('Frequency', size=15)

ax2.hist(scores['ENIGH'], bins=30, color='orange',edgecolor='purple', alpha=0.5)
ax2.set_xlabel('ENIGH', size=15)

ax3.hist(scores['Viviendas'], bins=30, color='blue',edgecolor='purple', alpha=0.5)
ax3.set_xlabel('Número de viviendas', size=15);

ax4.hist(scores['Poblacion'], bins=30, color='red',edgecolor='purple', alpha=0.5)
ax4.set_xlabel('Población', size=15)

ax5.hist(scores['Y objetivo'], bins=30, color='red',edgecolor='purple', alpha=0.5)
ax5.set_xlabel('Probabilidad de éxito', size=15)

# Dividimos los datos para entrenamiento y validación
Vemos que tenemos **105,952 registros y 36 columnas**, siendo una de las columnas nuestro valor objetivo ("probabilidad objetivo"). Lógicamente, utilizar la proporción convencional 80%-entrenamiento y  20%-validacion es un tanto excesivo así que **optaremos por utilizar 5,000 registros para validar y el resto será para entrenar a nuestra red neuronal.**

In [ ]:
# Selecciona solo las columnas que deseas dividir en conjuntos de entrenamiento y validación
data_to_split = scores.iloc[:, 4:]  # Esto excluye las primeras 4 columnas

# Revuelve aleatoriamente los datos
data_to_split = data_to_split.sample(frac=1, random_state=42)

# Divide los datos en conjunto de entrenamiento y validación
train_data, test_data = train_test_split(data_to_split, test_size=5000, random_state=42)

print("Tamaño datos de entrenamiento:", train_data.shape, "Tamaño datos de validación:", test_data.shape)

# Normalizamos nuestros datos de entrenamiento

$$x_{i,norm} = \dfrac{x_{i}-\mu}{\sigma}$$
    
$$y_{i,norm} = \dfrac{y_{i}-\mu}{\sigma}$$

In [ ]:
train_norm = (train_data - train_data.mean()) / train_data.std()
#Veamos una muestra de nuestro nuevo dataframe normalizado:
train_norm.head()

<font size=5 color='RED'>
Nota importante: La normalización de las muestras de prueba se realiza con los valores de $\mu$ y $\sigma$ obtenidos con las muestras empleadas para el entrenamiento

In [ ]:
test_norm = (test_data - train_data.mean()) / train_data.std()
test_norm.head()

# Separamos entre datos de entrada y la variable objetivo en ambos conjuntos

In [ ]:
x_train = train_norm.values[:,:-1]
y_train = train_norm.values[:,-1:]
print(type(x_train), type(y_train))
print(x_train.shape)
print(y_train.shape)

In [ ]:
x_validation = test_norm.values[:,:-1]
y_validation = test_norm.values[:,-1:]
print(type(x_validation), type(y_validation))
print(x_validation.shape)
print(y_validation.shape)

# Implementamos nuestra red neuronal

In [ ]:
from keras.layers import Dropout

# Define el modelo de la red neuronal
model = keras.Sequential()

# Capa de entrada con 36 unidades (una por cada columna en tus datos)
model.add(layers.Input(shape=(40,)))

# Capa oculta con 64 unidades y activación relu
model.add(layers.Dense(64, activation='relu'))
# Capa de Dropout con una tasa del 0.2 (20% de las neuronas se apagan aleatoriamente)
model.add(Dropout(0.2))

# Capa de salida con 1 unidad para regresión
model.add(layers.Dense(1, activation='linear'))

# Compila el modelo especificando la función de pérdida, el optimizador y las métricas
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mae'])

# Entrena el modelo con tus datos de entrenamiento y etiquetas
history = model.fit(train_norm, y_train, epochs=30, batch_size=32, validation_data=(test_norm, y_validation))

In [ ]:
# Evalúa el modelo en datos de validación (opcional)
validation_loss, validation_mae = model.evaluate(x_validation, y_validation)
print(f'Loss en datos de validación: {validation_loss}')
print(f'MAE en datos de validación: {validation_mae}')

# Realiza predicciones en nuevos datos (opcional)
predictions = model.predict(norm_validation_data)